# IBM Applied Data Science Capstone Course by Coursera
## Week 5 Final Report
### Opening a New Multiplex in Hyderabad , India

- Build a dataframe of neighborhoods in Hyderabad , India by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a multiplex

## Import libraries

In [64]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [7]:
data = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India').text
soup = BeautifulSoup(data, 'html.parser')

In [9]:
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [10]:
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
kl_df.head()

,Neighborhood
0,A. S. Rao Nagar
1,A.C. Guards
2,Abhyudaya Nagar
3,Abids
4,Adikmet


In [11]:
kl_df.shape

(200, 1)

In [17]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hyderabad, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [18]:
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [19]:
coords

[[17.455470000000048, 78.54389000000003],
 [17.392977027745946, 78.45686724051741],
 [17.337590000000034, 78.56235000000004],
 [17.389800000000037, 78.47658000000007],
 [17.410600000000045, 78.51515000000006],
 [17.37751000000003, 78.48005000000006],
 [17.3873489754401, 78.4669932273483],
 [17.346330000000023, 78.47255000000007],
 [17.360690000000034, 78.47996000000006],
 [17.51908000000003, 78.41435000000007],
 [17.535430000000076, 78.54427000000004],
 [17.385800000000074, 78.51841000000007],
 [17.435340000000053, 78.44861000000003],
 [17.407860000000028, 78.49151000000006],
 [17.38513000000006, 78.44738000000007],
 [17.369170000000054, 78.43683000000004],
 [17.407430000000033, 78.50220000000007],
 [17.372720000000072, 78.49047000000007],
 [17.38897000000003, 78.48681000000005],
 [17.399330000000077, 78.49965000000003],
 [17.341834499999997, 78.54754275],
 [17.448470000000043, 78.44937000000004],
 [17.41534000000007, 78.43437000000006],
 [17.386705398047635, 78.47824113930466],
 [17.3

In [27]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [28]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [31]:
print(kl_df.shape)
kl_df.head()

(200, 3)


,Neighborhood,Latitude,Longitude
0,A. S. Rao Nagar,17.455470,78.543890
1,A.C. Guards,17.392977,78.456867
2,Abhyudaya Nagar,17.337590,78.562350
3,Abids,17.389800,78.476580
4,Adikmet,17.410600,78.515150


In [32]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

In [33]:
# get the coordinates of Kuala Lumpur
address = 'Hyderabad, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hyderabad, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hyderabad, India 17.38878595, 78.4610647345315.


In [34]:
# create map of hyderabad using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [35]:
# save the map as HTML file
map_kl.save('map_kl.html')

In [40]:
# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: V4HK3Z2T3T1445O5ZBMJYA0CGWA4MLDNQSY03A2MWDUGZ2G3
CLIENT_SECRET:1SUC44AH5JNEADAIYG4I3IDX523VPASCUH5TT2KFD2ZE1U40


In [41]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [44]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6730, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,A. S. Rao Nagar,17.45547,78.54389,Cinepolis CCPL,17.457282,78.536823,Multiplex
1,A. S. Rao Nagar,17.45547,78.54389,Raghavendra Cine Complex,17.451299,78.535697,Indie Movie Theater
2,A. S. Rao Nagar,17.45547,78.54389,Reliance Fresh,17.456698,78.545802,Supermarket
3,A. S. Rao Nagar,17.45547,78.54389,Green Bawarchi,17.457901,78.543900,Indian Restaurant
4,A. S. Rao Nagar,17.45547,78.54389,gossips bakery,17.456999,78.545902,Fast Food Restaurant


In [45]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
A. S. Rao Nagar,16,16,16,16,16,16
A.C. Guards,52,52,52,52,52,52
Abhyudaya Nagar,7,7,7,7,7,7
Abids,82,82,82,82,82,82
Adikmet,16,16,16,16,16,16
Afzal Gunj,44,44,44,44,44,44
Aghapura,55,55,55,55,55,55
"Aliabad, Hyderabad",16,16,16,16,16,16
Alijah Kotla,17,17,17,17,17,17


In [46]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 166 uniques categories.


In [47]:
venues_df['VenueCategory'].unique()[:50]

array(['Multiplex', 'Indie Movie Theater', 'Supermarket',
       'Indian Restaurant', 'Fast Food Restaurant', 'Market', 'Tea Room',
       'Café', 'Neighborhood', 'Lake', 'Department Store',
       'Clothing Store', 'Train Station', 'Bakery',
       'Hyderabadi Restaurant', 'Lounge', 'Juice Bar',
       'South Indian Restaurant', 'Bistro', 'Park', 'Science Museum',
       'Ice Cream Shop', 'Middle Eastern Restaurant', 'Snack Place',
       'Hotel Bar', 'Vegetarian / Vegan Restaurant', 'Pizza Place',
       'Performing Arts Venue', 'Mobile Phone Shop', 'Stadium', 'Hotel',
       'Coffee Shop', 'Fried Chicken Joint', 'Hookah Bar',
       'Electronics Store', 'Platform', 'Bank', 'Restaurant',
       'Movie Theater', 'Food Truck', 'Diner', 'Shoe Store',
       'Chaat Place', 'Burger Joint', 'Dessert Shop', 'Food',
       'Chinese Restaurant', 'Shopping Mall', 'Smoke Shop', 'Bar'],
      dtype=object)

In [48]:
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [49]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(6730, 167)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Station,Business Service,Butcher,Cafeteria,Café,Candy Store,Castle,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Pool,Pub,Rajasthani Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [50]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(200, 167)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Station,Business Service,Butcher,Cafeteria,Café,Candy Store,Castle,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Pool,Pub,Rajasthani Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.062500,0.000000,0.0,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.125000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.00,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.062500,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.062500,0.00,0.00,0.000000,0.125000,0.000000,0.000000,0.00000,0.000000
1,A.C. Guards,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.057692,0.019231,0.000000,0.000000,0.000000,0.000000,0.000000,0.019231,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [65]:
len(kl_grouped[kl_grouped["Multiplex"] > 0])

68

In [66]:
kl_mall = kl_grouped[["Neighborhoods","Multiplex"]]
kl_mall.head()

,Neighborhoods,Multiplex
0,A. S. Rao Nagar,0.062500
1,A.C. Guards,0.000000
2,Abhyudaya Nagar,0.000000
3,Abids,0.012195
4,Adikmet,0.000000


In [54]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 2, 0, 0, 2, 0])

In [67]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [68]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Multiplex,Cluster Labels
0,A. S. Rao Nagar,0.062500,0
1,A.C. Guards,0.000000,0
2,Abhyudaya Nagar,0.000000,0
3,Abids,0.012195,0
4,Adikmet,0.000000,0


In [69]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(200, 5)


,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.062500,0,17.455470,78.543890
1,A.C. Guards,0.000000,0,17.392977,78.456867
2,Abhyudaya Nagar,0.000000,0,17.337590,78.562350
3,Abids,0.012195,0,17.389800,78.476580
4,Adikmet,0.000000,0,17.410600,78.515150


In [70]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(200, 5)


,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.062500,0,17.455470,78.543890
120,Manikonda,0.000000,0,17.401440,78.391660
121,Manovikas Nagar,0.000000,0,17.465610,78.493270
122,Marredpally,0.000000,0,17.447770,78.508730
123,Masab Tank,0.060000,0,17.400930,78.453630
125,Mehdipatnam,0.000000,0,17.392620,78.442210
126,Mettuguda,0.000000,0,17.434200,78.516360
127,"Minister Road, Hyderabad",0.000000,0,17.440273,78.479494
128,Mir Alam Tank,0.000000,0,17.355111,78.454120
129,Miyapur,0.000000,0,17.421020,78.582440


In [71]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [72]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [73]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.062500,0,17.455470,78.543890
120,Manikonda,0.000000,0,17.401440,78.391660
121,Manovikas Nagar,0.000000,0,17.465610,78.493270
122,Marredpally,0.000000,0,17.447770,78.508730
123,Masab Tank,0.060000,0,17.400930,78.453630
125,Mehdipatnam,0.000000,0,17.392620,78.442210
126,Mettuguda,0.000000,0,17.434200,78.516360
127,"Minister Road, Hyderabad",0.000000,0,17.440273,78.479494
128,Mir Alam Tank,0.000000,0,17.355111,78.454120
129,Miyapur,0.000000,0,17.421020,78.582440


In [74]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
15,Attapur,0.2,1,17.369170,78.436830
22,Bairamalguda,0.0,1,17.341834,78.547543
132,Moosapet,0.0,1,17.466990,78.428420
95,Kukatpally,0.0,1,17.487350,78.420870
162,Prakash Nagar,0.0,1,17.484860,78.417930
100,Lal Bahadur Nagar,0.1,1,17.377630,78.442370
187,Saroornagar,0.0,1,17.346650,78.538650
59,Gaddiannaram,0.0,1,17.351600,78.560710


In [75]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
16,"Azamabad, Hyderabad",0.000000,2,17.407430,78.502200
84,Karwan,0.062500,2,17.379120,78.436710
98,Lab quarters,0.035714,2,17.490517,78.391978
196,Silpa Avenue Colony,0.028571,2,17.485750,78.372220
97,Laad Bazaar,0.000000,2,17.361490,78.472080
130,Moazzam Jahi Market,0.018868,2,17.384480,78.474420
11,Amberpet,0.000000,2,17.385800,78.518410
193,Shivam Road,0.000000,2,17.402055,78.514052
131,Moghalpura,0.000000,2,17.358530,78.477070
8,Alijah Kotla,0.000000,2,17.360690,78.479960


## Observations:
Most of the Multiplexes are concentrated in the central area of Hyderabad city, with the highest number in cluster 0 and moderate number in cluster 2. On the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new multiplexes as there is very little to no competition from existing multiplexes. Meanwhile, Multiplexes in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of Multiplexes. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few multiplexes. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of multiplexes and suffering from intense competition.